# Generate set of administrative regions to use in exposure grid
- The idea here is to get a globally comprehensive set of administrative boundaries at the `adm1` level or higher if the `adm1` level is not available
- Use `adm1` if available and `adm0` if not

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import geopandas as gpd
import pandas as pd
from shapely.errors import ShapelyDeprecationWarning

from sliiders import settings as sset
from sliiders import spatial

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


#### Read full `adm0` and `adm1` sets

In [3]:
adm0 = (
    gpd.read_file(sset.PATH_GADM, layer="level0")[["GID_0", "geometry"]]
    .set_crs(epsg=4326)
    .set_index("GID_0")
    .geometry
)
adm1 = (
    gpd.read_file(sset.PATH_GADM, layer="level1")[["GID_0", "GID_1", "geometry"]]
    .set_crs(epsg=4326)
    .set_index("GID_1")
)

#### Exclude non-territory ISOs
- `ATA`: Antarctica
- `XCA`: Caspian Sea

In [4]:
sset.EXCLUDED_ISOS

['ATA', 'XCA']

In [5]:
# Filter out excluded ISOs
adm0 = adm0[~adm0.index.isin(sset.EXCLUDED_ISOS)]
adm1 = adm1[~adm1["GID_0"].isin(sset.EXCLUDED_ISOS)]

#### Select only countries in `adm0` that do not appear in the `adm1` set, to be included as intermediate `adm1` regions

In [6]:
adm0_as_adm1 = adm0.drop(adm1.GID_0.unique())

#### Concatenate all `adm1` regions

In [7]:
full = pd.concat([adm0_as_adm1.to_frame(), adm1]).geometry.to_frame()
full["ISO"] = full.index.str.split(".").str[0]
full.index.name = "adm1"
assert full.index.is_unique
seg_centroids = gpd.read_file(sset.PATH_SEG_CENTROIDS).set_index("station_id").geometry
coastlines = gpd.read_file(sset.PATH_CIAM_COASTLINES).set_index("line_id").geometry
overlay_name = "seg_adm"

In [13]:
seg_centroids

station_id
id_coast_glob_eur_00001       POINT (24.08936 34.82666)
id_coast_glob_eur_00002       POINT (24.08936 34.87061)
id_coast_glob_eur_00008       POINT (12.57568 35.50049)
id_coast_glob_eur_00009       POINT (12.61963 35.50049)
id_coast_glob_eur_00167       POINT (23.32764 35.83740)
                                       ...             
id_coast_glob_99011        POINT (-128.32974 -24.38661)
id_coast_glob_99012          POINT (115.80228 10.73177)
id_coast_glob_99013         POINT (-171.19264 -9.34152)
id_coast_glob_99014          POINT (166.64951 19.28119)
id_coast_glob_99015          POINT (-63.01482 18.03886)
Name: geometry, Length: 9592, dtype: geometry

In [11]:
spatial.append_extra_pts(full.head())

ValueError: y attribute access only provided for Point geometries

In [8]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
    all_overlays, ciam_polys = spatial.create_overlay_voronois(
        full, seg_centroids, coastlines, overlay_name, plot=True
    )

Generating global Voronoi shapes for regions...


  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/1812 [00:00<?, ?it/s]

  0%|          | 0/1812 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Assigning countries to segment centroids...
Generating within-country Voronoi shapes for segment centroids...
ABW AGO AIA ALA ALB ARE ARG ASM ATF ATG AUS BEL 

AttributeError: 'GeoDataFrame' object has no attribute 'y'

In [ ]:
sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS.parent.mkdir(exist_ok=False)

all_overlays.to_parquet(sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS, index=False)
all_overlays.to_file(sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS_SHP)

ciam_polys.to_parquet(sset.PATH_CIAM_POLYS, index=False)